In [1]:
import pandas as pd
import requests

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Access the key
API_KEY = os.getenv("FBR_API_KEY")

if not API_KEY:
    raise ValueError("⚠️ Missing FBR_API_KEY in your .env file!")

print("✅ API key loaded successfully!")

headers = {"X-API-Key": API_KEY}

✅ API key loaded successfully!


In [3]:
# Premiere League ID is 9
response = requests.get("https://fbrapi.com/league-seasons", params={"league_id": 9}, headers=headers)
data = response.json()
records = data['data']
pl_seasons = pd.json_normalize(records)
pl_seasons

,season_id,competition_name,#_squads,champion,top_scorer.player,top_scorer.goals_scored
0,2025-2026,Premier League,20,,Erling Haaland,9.0
1,2024-2025,Premier League,20,Liverpool,Mohamed Salah,29.0
2,2023-2024,Premier League,20,Manchester City,Erling Haaland,27.0
3,2022-2023,Premier League,20,Manchester City,Erling Haaland,36.0
4,2021-2022,Premier League,20,Manchester City,"[Son Heung min, Mohamed Salah]",23.0
...,...,...,...,...,...,...
122,1892-1893,Football League First Division,16,Sunderland,,NaN
123,1891-1892,Football League,14,Sunderland,,NaN
124,1890-1891,Football League,12,Everton,,NaN
125,1889-1890,Football League,12,Preston North End,,NaN


In [38]:
# Not very useful. Data from teams endpoint is better.

response = requests.get("https://fbrapi.com/matches", params={"league_id": 9, "season_id": "2024-2025"}, headers=headers)
data = response.json()
records = data['data']
matches = pd.json_normalize(records)

In [ ]:
# tms = matches[['home', 'home_team_id']].drop_duplicates(subset=['home', 'home_team_id']).reset_index(drop=True)
# teams = pd.read_csv('../data/pl_teams.csv')
# teams = teams.rename(columns={'team': 'home', 'team_id': 'home_team_id'})
# teams = pd.concat([teams, tms], ignore_index=True, axis=0).drop_duplicates(subset=['home_team_id']).reset_index(drop=True)
# teams.to_csv('../data/pl_teams.csv', index=False)
# teams

,home,home_team_id
0,Aston Villa,8602292d
1,Wolves,8cec06e1
2,Manchester City,b8fd03ef
3,Newcastle Utd,b2b47a98
4,Brentford,cd051869
5,Crystal Palace,47c64c55
6,Nott'ham Forest,e4a775cb
7,Burnley,943e8050
8,Everton,d3fd31cc
9,Brighton,d07537b9


In [ ]:
# pl_matches['date'] = pd.to_datetime(pl_matches['date'])
# pl_matches = pl_matches[pl_matches['date'] <= pd.Timestamp.now()].sort_values(by='date', ascending=False)
# pl_matches

,match_id,date,time,wk,home,home_team_id,away,away_team_id,home_team_score,away_team_score,venue,attendance,referee
69,8c0ee359,2025-10-05,16:30,7,Brentford,cd051869,Manchester City,b8fd03ef,None,None,Gtech Community Stadium,"17,028",Darren England
68,e33f1657,2025-10-05,14:00,7,Newcastle Utd,b2b47a98,Nott'ham Forest,e4a775cb,None,None,St James' Park,"52,189",Peter Bankes
67,db870461,2025-10-05,14:00,7,Aston Villa,8602292d,Burnley,943e8050,None,None,Villa Park,"40,308",Andrew Kitchen
66,88e53add,2025-10-05,14:00,7,Everton,d3fd31cc,Crystal Palace,47c64c55,None,None,Hill Dickinson Stadium,"51,770",Michael Salisbury
65,1314f9fd,2025-10-05,14:00,7,Wolves,8cec06e1,Brighton,d07537b9,None,None,Molineux Stadium,"29,504",Jarred Gillett
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,e1058522,2025-08-16,15:00,1,Sunderland,8ef52968,West Ham,7c21e445,None,None,Stadium of Light,"46,233",Robert Jones
1,bbdf4739,2025-08-16,12:30,1,Aston Villa,8602292d,Newcastle Utd,b2b47a98,None,None,Villa Park,"42,526",Craig Pawson
4,d41bb8b3,2025-08-16,15:00,1,Tottenham,361ca564,Burnley,943e8050,None,None,Tottenham Hotspur Stadium,"61,077",Michael Oliver
5,8613020d,2025-08-16,17:30,1,Wolves,8cec06e1,Manchester City,b8fd03ef,None,None,Molineux Stadium,"31,456",Jarred Gillett


In [41]:
teams = pd.read_csv('../data/pl_teams.csv')
tms = teams['home'].unique()
ids = teams['home_team_id'].unique()

teams = {}
for i in range(len(tms)):
    teams[tms[i]] = ids[i]
print(teams)

{'Aston Villa': '8602292d', 'Wolves': '8cec06e1', 'Manchester City': 'b8fd03ef', 'Newcastle Utd': 'b2b47a98', 'Brentford': 'cd051869', 'Crystal Palace': '47c64c55', "Nott'ham Forest": 'e4a775cb', 'Burnley': '943e8050', 'Everton': 'd3fd31cc', 'Brighton': 'd07537b9', 'Chelsea': 'cff3d9bb', 'West Ham': '7c21e445', 'Liverpool': '822bd0ba', 'Leeds United': '5bfb9659', 'Sunderland': '8ef52968', 'Tottenham': '361ca564', 'Arsenal': '18bb7c10', 'Manchester Utd': '19538871', 'Fulham': 'fd962109', 'Bournemouth': '4ba7cbea', 'Sheffield Utd': '1df6b87e', 'Luton Town': 'e297cd13', 'Ipswich Town': 'b74092de', 'Leicester City': 'a2d435b3', 'Southampton': '33c895d4'}


In [ ]:
# pl_matches = pd.read_csv('../data/pl_matches.csv')

In [46]:
# Gather matches from each team

import time
import random

for team in teams.keys():
    response = requests.get("https://fbrapi.com/teams", params={"team_id": teams[team], "season_id": "2023-2024"}, headers=headers)
    data = response.json()
    records = data['team_schedule']['data']
    df = pd.json_normalize(records)
    df['home_team'] = team
    df['home_team_id'] = teams[team]
    pl_matches = pd.concat([pl_matches, df[df['league_id'] == 9]], ignore_index=True)
    time.sleep(random.uniform(3, 7)) # Sleep between 3 to 7 seconds to avoid rate limiting

In [47]:
pl_matches['date'] = pd.to_datetime(pl_matches['date'])
pl_matches = pl_matches[pl_matches['date'] <= pd.Timestamp.now()].sort_values(by='date', ascending=False)
pl_matches.reset_index(drop=True, inplace=True)
pl_matches
#pl_matches.to_csv('../data/pl_matches.csv', index=False)

,date,time,match_id,league_name,league_id,opponent,opponent_id,home_away,result,gf,ga,attendance,captain,formation,referee,home_team,home_team_id
0,2025-10-05,16:30,8c0ee359,Premier League,9,Brentford,cd051869,Away,W,1.0,0.0,"17,028",Rúben Dias,4-1-4-1,Darren England,Manchester City,b8fd03ef
1,2025-10-05,14:00,db870461,Premier League,9,Aston Villa,8602292d,Away,L,1.0,2.0,"40,308",Josh Cullen,5-4-1,Andrew Kitchen,Burnley,943e8050
2,2025-10-05,14:00,88e53add,Premier League,9,Crystal Palace,47c64c55,Home,W,2.0,1.0,"51,770",James Tarkowski,4-2-3-1,Michael Salisbury,Everton,d3fd31cc
3,2025-10-05,14:00,1314f9fd,Premier League,9,Brighton,d07537b9,Home,D,1.0,1.0,"29,504",João Gomes,4-3-3,Jarred Gillett,Wolves,8cec06e1
4,2025-10-05,14:00,1314f9fd,Premier League,9,Wolves,8cec06e1,Away,D,1.0,1.0,"29,504",Lewis Dunk,4-2-3-1,Jarred Gillett,Brighton,d07537b9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,2023-08-12,15:00,56a137f7,Premier League,9,Brighton,d07537b9,Away,L,1.0,4.0,"31,872",Tom Lockyer,3-5-2,David Coote,Luton Town,e297cd13
1656,2023-08-12,15:00,55fd92c7,Premier League,9,Sheffield Utd,1df6b87e,Away,W,1.0,0.0,"31,194",Joel Ward,4-2-3-1,John Brooks,Crystal Palace,47c64c55
1657,2023-08-12,15:00,15addfc7,Premier League,9,Fulham,fd962109,Home,L,0.0,1.0,"39,940",James Tarkowski,4-4-1-1,Stuart Attwell,Everton,d3fd31cc
1658,2023-08-11,20:00,3a6836b4,Premier League,9,Burnley,943e8050,Away,W,3.0,0.0,"21,572",Kevin De Bruyne,4-2-3-1,Craig Pawson,Manchester City,b8fd03ef


In [52]:
# Remove duplicates
pl_matches = pl_matches.drop_duplicates(subset=['match_id'], keep='first')
pl_matches.value_counts('match_id').sort_values(ascending=False)

pl_matches.to_csv('../data/pl_matches.csv', index=False)